In [64]:
# Project
# github restricts file size, dataset can be downloaded from https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics/data


In [65]:
# import the dataset
import io
import pandas as pd

data = pd.read_csv('lyrics.csv')
data.head()

# temp so that we speed up the testing
data=data.head()  

In [80]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

# creates a bag of words
X = vectorizer.fit_transform(data.lyrics)

In [81]:
# visualize the transformed data 
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())